In [15]:
import pandas as pd
from datetime import datetime
import time
from tabulate import tabulate
import concurrent.futures

class StockTrader:
    def __init__(self, initial_balance=100000):
        self.balance = initial_balance
        self.stock_quantity = 0
        self.stock_price = 0
        self.last_generated_id = int(time.time())
        self.exit_reason = 'Manual'
        columns = ['Symbol', 'Side','Status', 'BuyID', 'Quantity', 'BuyPrice', 'BuyDatetime', 'SellTransactions',
                   'PendingQuantity', 'AverageSellPrice', 'PnL', 'SLValue', 'TargetValue', 'ExitReason']
        self.logs = pd.DataFrame(columns=columns)
    def generate_id(self):
        unique_id = self.last_generated_id
        self.last_generated_id += 1
        return unique_id

    def buy_stock(self, quantity, price, date, symbol,side):
        buy_id = self.generate_id()
        buy_datetime = datetime.strptime(date, '%Y-%m-%d')
        self.stock_quantity += quantity
        self.stock_price = price
        
        if side == "CE":
            self.stoploss = price -20
            self.target = price + 40
        elif side == "PE":
            self.stoploss = price +20
            self.target = price - 40
        else:
            print("SOMETHING WRONG IS SIDE SELECTION")

        new_log = pd.DataFrame({'Symbol': [symbol],'Side':[side] ,'BuyID': [buy_id], 'Quantity': [quantity], 'BuyPrice': [price],
                                'BuyDatetime': [buy_datetime], 'SellTransactions': [[]], 'PendingQuantity': [quantity],
                                'AverageSellPrice': [0], 'PnL': [0], 'Status': ['Open'],
                                'SLValue': [self.stoploss], 'TargetValue': [self.target], 'ExitReason': [self.exit_reason]})
        self.logs = pd.concat([self.logs, new_log], ignore_index=True)
        return buy_id

    def sell_stock(self, buy_id, quantity, sell_price, date, exit_reason='Manual'):
        buy_row = self.logs[self.logs['BuyID'] == buy_id]
        if buy_row.empty:
            return "Buy ID not found."
            
        sell_datetime = datetime.strptime(date, '%Y-%m-%d')
        tred_side = buy_row['Side'].iloc[0]
        if tred_side == "CE":
            pnl = quantity * (sell_price - buy_row['BuyPrice'].values[0])
            print("pnl",pnl)

        else:
            pnl = quantity * (buy_row['BuyPrice'].values[0] - sell_price)
            print("pnl",pnl)
 
            
        # Update the log entry with the sell transaction
        self.balance += pnl
        buy_index = buy_row.index[0]
        sell_transaction = {'Quantity': quantity, 'SellPrice': sell_price, 'SellDatetime': sell_datetime, 'PnL': pnl}
        self.logs.at[buy_index, 'SellTransactions'] = self.logs.at[buy_index, 'SellTransactions'] + [sell_transaction]

        # Update pending quantity, average sell price, and cumulative PnL
        self.logs.at[buy_index, 'PendingQuantity'] -= quantity
        self.logs.at[buy_index, 'AverageSellPrice'] = self.calculate_average_sell_price(buy_index)
        self.logs.at[buy_index, 'PnL'] += pnl

        # If all quantity sold, update the status to 'Done'
        if self.logs.at[buy_index, 'PendingQuantity'] == 0:
            self.logs.at[buy_index, 'Status'] = 'Done'

        # Set exit reason and update SLValue and TargetValue
        self.exit_reason = exit_reason
        self.logs.at[buy_index, 'SLValue'] = self.stoploss
        self.logs.at[buy_index, 'TargetValue'] = self.target
        self.logs.at[buy_index, 'ExitReason'] = exit_reason

        return f"Stock sold successfully. PnL: {pnl}"

        
    def calculate_average_sell_price(self, buy_index):
        sell_transactions = self.logs.at[buy_index, 'SellTransactions']
        if not sell_transactions:
            return 0

        total_sell_price = sum(transaction['Quantity'] * transaction['SellPrice'] for transaction in sell_transactions)
        total_quantity = sum(transaction['Quantity'] for transaction in sell_transactions)

        return total_sell_price / total_quantity



    def auto_exit_with_trailing(self, list_of_symbol):
        self.trade_executed = False
        self.trailing_sl = 10
        self.trailing_target = 15

        def process_symbol(i):
            symbol = i['Symbol']
            current_price = i['Price']
        
            ce_entries = self.logs[(self.logs['Side'] == 'CE') & (self.logs['SLValue'] >= current_price) & (self.logs['Symbol'] == symbol)]
            pe_entries = self.logs[(self.logs['Side'] == 'PE') & (self.logs['SLValue'] <= current_price) & (self.logs['Symbol'] == symbol)]
            relevant_entries_ls = pd.concat([ce_entries, pe_entries])
        
            for buy_index, buy_row in relevant_entries_ls.iterrows():
                buying_price = buy_row['BuyPrice']
                qnty = buy_row['PendingQuantity']
                buy_id = buy_row['BuyID']
        
                if qnty != 0:
                    percentage_loss = ((buying_price - current_price) / buying_price) * 100
                    exit_reason = "Trailing SL"
                    
                    if buy_row['Side'] == "CE" and current_price < buy_row['SLValue']:
                        print(f"SL EXIT DONE [{buy_id}] LOSS% : {percentage_loss:.2f}%")
                        self.sell_stock(buy_id=buy_id, quantity=qnty, sell_price=current_price, date=time.strftime('%Y-%m-%d'), exit_reason=exit_reason)
                        
                    elif buy_row['Side'] == "PE" and current_price > buy_row['SLValue']:
                        print(f"SL EXIT DONE [{buy_id}] LOSS% : {percentage_loss:.2f}%")
                        self.sell_stock(buy_id=buy_id, quantity=qnty, sell_price=current_price, date=time.strftime('%Y-%m-%d'), exit_reason=exit_reason)
        
            relevant_entries_target = self.logs[self.logs['Symbol'] == symbol]
        
            for buy_index, buy_row in relevant_entries_target.iterrows():
                buy_id = buy_row['BuyID']
        
                if buy_row['Side'] == "CE" and current_price > buy_row['TargetValue']:
                    print(f"TRAILING TARGET ACHIEVED [{buy_id}]")
                    new_sl = round((buy_row['TargetValue'] - buy_row['TargetValue'] * (self.trailing_sl / 100)), 2)
                    new_target = round((buy_row['TargetValue'] + buy_row['TargetValue'] * (self.trailing_target / 100)), 2)
                    self.logs.at[buy_index, 'SLValue'] = new_sl
                    self.logs.at[buy_index, 'TargetValue'] = new_target

                elif buy_row['Side'] == "PE" and current_price <= buy_row['TargetValue']:
                    print(f"TRAILING TARGET ACHIEVED [{buy_id}]")
                    new_sl = round((buy_row['TargetValue'] + buy_row['TargetValue'] * (self.trailing_sl / 100)), 2)
                    new_target = round((buy_row['TargetValue'] - buy_row['TargetValue'] * (self.trailing_target / 100)), 2)
                    self.logs.at[buy_index, 'SLValue'] = new_sl
                    self.logs.at[buy_index, 'TargetValue'] = new_target
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(process_symbol, list_of_symbol)


            
    
    def stats(self):
            df = self.logs
            total_trade = len(df.index)
            pnl = df.PnL.sum()
            winners = len(df[df.PnL > 0])
            losers = len(df[df.PnL <= 0])
            win_ratio = round((winners / total_trade) * 100, 2)
    
            # Calculate additional metrics
            capital = self.balance
            max_win = round(df[df.PnL > 0].PnL.max(), 2)
            max_loss = round(df[df.PnL <= 0].PnL.min(), 2)
            total_profit = round(df.PnL.sum(), 2)
            total_profit_percentage = round((total_profit / self.balance) * 100, 2)
    
            # Prepare the data for tabular representation
            parameters = ['Total Trades', 'Capital', 'Total Wins', 'Total Losses', 'Win Ratio',
                          'Max Win', 'Max Loss', 'Total Profit', ' Grow Profit %']
            data_points = [total_trade, capital, winners, losers, f"{win_ratio}%",
                           max_win, max_loss, total_profit, f"{total_profit_percentage}%"]
            data = list(zip(parameters, data_points))

            # Print the tabular representation
            print(tabulate(data, headers=['Parameters', 'Values'], tablefmt='psql'))

In [16]:
trader = StockTrader()

In [17]:
# buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "IPL",side = "PE")
# buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "IPL",side = "PE")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "NJ",side = "CE")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "NJ",side = "CE")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "NJ",side = "CE")
buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "NJ",side = "CE")
# buy_id_1 = trader.buy_stock(quantity=50, price=60, date='2023-01-01',symbol = "SMP",side = "CE")
# sell_result_1 = trader.sell_stock(buy_id=buy_id_1, quantity=10, sell_price=500, date='2023-02-01')

In [20]:
trader.auto_exit_with_trailing([{"Symbol":"NJ","Price": 10 }])

SL EXIT DONE [1703932135] LOSS% : 83.33%
pnl -2500
SL EXIT DONE [1703932136] LOSS% : 83.33%
pnl -2500
SL EXIT DONE [1703932137] LOSS% : 83.33%
pnl -2500
SL EXIT DONE [1703932138] LOSS% : 83.33%
pnl -2500


In [21]:
# trader.stats()
trader.logs

,Symbol,Side,Status,BuyID,Quantity,BuyPrice,BuyDatetime,SellTransactions,PendingQuantity,AverageSellPrice,PnL,SLValue,TargetValue,ExitReason
0,NJ,CE,Done,1703932135,50,60,2023-01-01,"[{'Quantity': 50, 'SellPrice': 10, 'SellDateti...",0,10.0,-2500,40,100,Trailing SL
1,NJ,CE,Done,1703932136,50,60,2023-01-01,"[{'Quantity': 50, 'SellPrice': 10, 'SellDateti...",0,10.0,-2500,40,100,Trailing SL
2,NJ,CE,Done,1703932137,50,60,2023-01-01,"[{'Quantity': 50, 'SellPrice': 10, 'SellDateti...",0,10.0,-2500,40,100,Trailing SL
3,NJ,CE,Done,1703932138,50,60,2023-01-01,"[{'Quantity': 50, 'SellPrice': 10, 'SellDateti...",0,10.0,-2500,40,100,Trailing SL
